# Automate the Grantee Website Monitoring

In [1]:
#Load Packages
import requests
import csv
from bs4 import BeautifulSoup
from googlesearch import search
#from urllib.request import urlopen
#from urllib.parse import urljoin
import urllib.request
import urllib
import pathlib
from time import sleep
import pandas as pd
import progressbar
import os

## Step 1: Initialize data frame

In [42]:
wd = os.getcwd()

# Import list of schools 
#columnNames = ['School Name', 'Status','Number Active URL', 'Number PDFs', 'Site', 'Google Queried', 'Google Reports Found', 'Reports Downloaded']
data = pd.read_csv('./all 5000 statuses.csv') #, names=columnNames)
#print(df)
df = data[['school_name']]

# Extract general school website from URL file and add to data frame
# Read in provided URLS
file = pd.read_excel('./IHE Student URLS January.xlsm')
#raw_url = file['URL']
df = df.join(file)

# Extract primary component of URL
#urldf = pd.DataFrame(file['URL']) #, dtype="string")
#df['rawURL'] = raw_url
df['shortURL']= df.URL.str.split("/",expand=True)[2]

# Remove preceding "www." if present
df['Site'] = [str(x).replace('www.','') for x in df['shortURL']]

#df = df.set_index('index')
df = df[["school_name","OPE ID", "Site"]]
df = df.head(10)
print(df)

                          school_name     OPE ID                     Site
0                ALASKA BIBLE COLLEGE   884300.0              akbible.edu
1               ALASKA CAREER COLLEGE  2541000.0  alaskacareercollege.edu
2            ALASKA CHRISTIAN COLLEGE  4138600.0             alaskacc.edu
3           ALASKA PACIFIC UNIVERSITY   106100.0        alaskapacific.edu
4  ALASKA VOCATIONAL TECHNICAL CENTER        NaN                avtec.edu
5                   ILISAGVIK COLLEGE  3461300.0            ilisagvik.edu
6      UNIVERSITY OF ALASKA ANCHORAGE        NaN           uaa.alaska.edu
7      UNIVERSITY OF ALASKA FAIRBANKS        NaN                  uaf.edu
8      UNIVERSITY OF ALASKA SOUTHEAST   106500.0           uas.alaska.edu
9              ALABAMA A&M UNIVERSITY        NaN                 aamu.edu


## Step 2: Search Google for PDFs

In [45]:
# EXAMPLE query = " \"Institutional Portion\" HEERF filetype:pdf site:franu.edu"
df['Query'] = df['Site'].apply(lambda x: "\"Institutional Portion\" HEERF filetype:pdf site:" + x)

def google_search(siteStr):
    results = []
    query = "\"Institutional Portion\" HEERF filetype:pdf site:" + siteStr
    df['Google Queried'] = 'Y' 
    for j in search(query, tld="co.in", num=10, stop = None, pause = 4):
        #count = count + 1
        #print(j)
        results.append(j)
        count = len(results)
    return results

df['Results'] = df['Site'].apply(lambda x: google_search(x))
df['Number of Results'] = df['Results'].str.len()

df.head()

,school_name,OPE ID,Site,Query,Google Queried,Results,Number of Results
0,ALASKA BIBLE COLLEGE,884300.0,akbible.edu,"""Institutional Portion"" HEERF filetype:pdf sit...",Y,[https://www.akbible.edu/wp-content/uploads/20...,1
1,ALASKA CAREER COLLEGE,2541000.0,alaskacareercollege.edu,"""Institutional Portion"" HEERF filetype:pdf sit...",Y,[https://www.alaskacareercollege.edu/uploads/d...,3
2,ALASKA CHRISTIAN COLLEGE,4138600.0,alaskacc.edu,"""Institutional Portion"" HEERF filetype:pdf sit...",Y,[https://alaskacc.edu/wp-content/uploads/2021/...,1
3,ALASKA PACIFIC UNIVERSITY,106100.0,alaskapacific.edu,"""Institutional Portion"" HEERF filetype:pdf sit...",Y,[https://www.alaskapacific.edu/wp-content/uplo...,1
4,ALASKA VOCATIONAL TECHNICAL CENTER,NaN,avtec.edu,"""Institutional Portion"" HEERF filetype:pdf sit...",Y,[https://avtec.edu/sites/default/files/2021-07...,2


In [85]:
df.Results.dtypes
y = df.Results.values
print(y[0])

['https://www.akbible.edu/wp-content/uploads/2020/10/heerf-quarterly-reporting-v131-1.pdf']


## Step 3: Download PDFs and save in local folder

In [ ]:
#urllib.request.urlretrieve(url, "test.pdf")
def makedirectory(name):
    try:
        os.makedirs(name)
    except OSError:
        pass
    os.chdir(wd)
    
def downloadReports(name):
    print(name)
    #name=str(name)
    os.chdir(name)
    url_list = df.loc[df.school_name == name]['Results'].values
    #print(url_list)
    imax = len(url_list) - 1
    for i in (0,imax):
        url = url_list.values
        print(url)
        filename = "test" + str(i) + ".pdf"
        # use the pdf name "Quarterly Grand Reporting" XXX_1
        urllib.request.urlretrieve(url, filename)
    os.chdir(wd)

df['school_name'].apply(makedirectory)
#df['school_name'].apply(downloadReports)
os.chdir(wd)

#print(url_list)
#print(url)

In [91]:
df.head()
name = "ALASKA VOCATIONAL TECHNICAL CENTER"
url_list = df.loc[df.school_name == name]['Results'].values
for url in url_list[0]:
    urllib.request.urlretrieve(url,  "test.pdf")
    print(url)

https://avtec.edu/sites/default/files/2021-07/Quarterly%20Grant%20Reporting%206-30-2021_1.pdf
https://avtec.edu/sites/default/files/2021-04/Quarterly%20Grant%20Reporting%203-31-2021.pdf


In [ ]:
os.chdir(wd)
cwd = os.getcwd()
print(cwd)

In [ ]:
name = df['school_name'][0]
#temp = df.query("school_name = name")
temp = df.loc[df.school_name == name]['Results']
print(temp)